In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
import optuna

In [ ]:
BASE_PATH = '/kaggle/input/aptos2019/'
TRAIN_CSV = os.path.join(BASE_PATH, 'train_1.csv')
VAL_CSV = os.path.join(BASE_PATH, 'valid.csv')
TRAIN_DIR = os.path.join(BASE_PATH, 'train_images/train_images')
VAL_DIR = os.path.join(BASE_PATH, 'val_images/val_images')

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 10
MIXUP_ALPHA = 1.0 # Alpha for Mixup, 1.0 is a good default
MODEL_SAVE_PATH = 'best_model.pth'

In [ ]:
# 3. --- AUGMENTATIONS ---
# A robust set of augmentations for this task
train_transform = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.CLAHE(p=0.4), # Enhance local contrast
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
# 4. --- PYTORCH DATASET CLASS ---
class DRDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name = self.df.iloc[index]['id_code'] + '.png'
        label = self.df.iloc[index]['diagnosis']
        img_path = os.path.join(self.image_dir, img_name)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)['image']

        return image, label


In [ ]:
# 5. --- HYBRID MODEL DEFINITION ---
class HybridModel(nn.Module):
    def __init__(self, num_classes=5, dropout_rate=0.5):
        super(HybridModel, self).__init__()
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True, num_classes=0)
        self.resnet = timm.create_model('resnet18', pretrained=True, num_classes=0)
        self.fusion = nn.Sequential(
            nn.Linear(self.efficientnet.num_features + self.resnet.num_features, 512),
            nn.ReLU(),
            nn.Dropout(dropout_rate), # Use the passed dropout rate
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        feat1 = self.efficientnet(x)
        feat2 = self.resnet(x)
        combined = torch.cat((feat1, feat2), dim=1)
        out = self.fusion(combined)
        return out

In [ ]:
# 6. --- HELPER FUNCTION FOR MIXUP LOSS ---
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


In [ ]:
# 7. --- OPTUNA OBJECTIVE FUNCTION ---
# <-- NEW: This function wraps the entire training and validation process for one trial -->
def objective(trial):
    # --- 7.1 Suggest Hyperparameters ---
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.6)
    mixup_alpha = trial.suggest_float("mixup_alpha", 0.1, 1.0)

    # --- 7.2 Model, Optimizer, and Loss ---
    model = HybridModel(num_classes=5, dropout_rate=dropout_rate).to(DEVICE)
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # --- 7.3 Training & Validation Loop ---
    for epoch in range(EPOCHS):
        # Training phase
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            # Mixup
            lam = np.random.beta(mixup_alpha, mixup_alpha)
            rand_index = torch.randperm(images.size()[0])
            mixed_images = lam * images + (1 - lam) * images[rand_index]
            labels_b = labels[rand_index]

            optimizer.zero_grad()
            outputs = model(mixed_images)
            loss = mixup_criterion(criterion, outputs, labels, labels_b, lam)
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(all_labels, all_preds)

        # --- 7.4 Report to Optuna for Pruning ---
        trial.report(val_accuracy, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # Return the final validation accuracy for the trial
    return val_accuracy


In [ ]:
# 8. --- MAIN SCRIPT EXECUTION ---
if __name__ == '__main__':
    # Load dataframes and create DataLoaders once to be used by all trials
    train_df = pd.read_csv(TRAIN_CSV)
    val_df = pd.read_csv(VAL_CSV)
    train_dataset = DRDataset(train_df, TRAIN_DIR, transform=train_transform)
    val_dataset = DRDataset(val_df, VAL_DIR, transform=val_transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    # --- Create and run the Optuna study ---
    print(f"--- Starting Hyperparameter Tuning with Optuna ---")
    # We want to maximize accuracy, and we use a pruner to stop bad trials early.
    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())

    # Start the optimization. Optuna will run the 'objective' function for 25 trials.
    study.optimize(objective, n_trials=25)

    # --- Print the results ---
    print("\n--- Tuning Complete ---")
    print(f"Number of finished trials: {len(study.trials)}")

    print("\nBest trial:")
    trial = study.best_trial

    print(f"  Value (Validation Accuracy): {trial.value:.4f}")

    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

[I 2025-08-19 13:11:16,666] A new study created in memory with name: no-name-3ffe1fa5-bde4-426e-b480-d7c3e6831403


--- Starting Hyperparameter Tuning with Optuna ---


[I 2025-08-19 13:42:48,385] Trial 0 finished with value: 0.8060109289617486 and parameters: {'lr': 0.0006924724408386977, 'optimizer': 'RMSprop', 'dropout_rate': 0.4360762843349014, 'mixup_alpha': 0.2382602128545406}. Best is trial 0 with value: 0.8060109289617486.
[I 2025-08-19 14:14:13,152] Trial 1 finished with value: 0.4207650273224044 and parameters: {'lr': 2.294832229632033e-05, 'optimizer': 'SGD', 'dropout_rate': 0.23806226661054575, 'mixup_alpha': 0.6381745549998576}. Best is trial 0 with value: 0.8060109289617486.
[I 2025-08-19 14:45:44,063] Trial 2 finished with value: 0.8169398907103825 and parameters: {'lr': 0.0005230504404111168, 'optimizer': 'Adam', 'dropout_rate': 0.436103647536376, 'mixup_alpha': 0.5903698982696499}. Best is trial 2 with value: 0.8169398907103825.
[I 2025-08-19 15:17:03,978] Trial 3 finished with value: 0.8306010928961749 and parameters: {'lr': 0.0005931894131265586, 'optimizer': 'Adam', 'dropout_rate': 0.536617352293986, 'mixup_alpha': 0.31800954961038